In [ ]:
library(tidyverse)
library(osmdata)
library(sf)
library(terra)
library(leaflet)

In [ ]:
# Get Kiambu
kmb <- opq(bbox = 'Kiambu KE') %>%
  add_osm_feature(key = 'name', value = "Kiambu", value_exact = T) %>%
  add_osm_feature(key = 'admin_level', value = "4", value_exact = T) %>%
  osmdata_sf()
# Get Nairobi
nrb <- opq(bbox = 'Nairobi KE') %>%
  add_osm_feature(key = 'name', value = "Nairobi", value_exact = T) %>%
  add_osm_feature(key = 'admin_level', value = "4", value_exact = T) %>%
  osmdata_sf()
# Merge Kiambu & Nairobi
aoi <- st_union(kmb$osm_multipolygons, nrb$osm_multipolygons)
# Plot the AOI
leaflet() %>%
  addTiles() %>%
  addPolygons(data = aoi$geometry, weight = 2, fillColor = NULL)

In [ ]:
## Get NASA's weather data (downloaded from CG Labs using NASA POWER API)
weather.JSON <- rjson::fromJSON(file = "POWER_Regional_Daily_20170201_20210207_74d3dbce.json")
## Start formatting the JSON data
weather.JSON <- enframe(unlist(weather.JSON))
## Begin formatting the JSON data
weather.JSON <- weather.JSON %>% 
  separate(name, into = c(paste0("name", 1:5)), fill = "right") %>%
  ## Here filter columns containing the following strings
  filter(name1 == "features" &
           name2 == "geometry" &
           name3 %in% c("coordinates1", "coordinates2") |
           (name1 == "features" &
              name2 == "properties" &
              name3 == "parameter")) %>%
  ## Only keep these two columns
  select(name5, value)
## Create new columns
weather.JSON <- weather.JSON %>%
  mutate(lat = if_else(is.na(name5), as.numeric(value), as.numeric(NA))) %>%
  mutate(lng = if_else(is.na(name5), as.numeric(lag(value, n = 1L)), as.numeric(NA))) %>% # Use the cell above as Longitude... This has to do with the way the JSON is structured
  mutate(date = if_else(is.na(name5), as.Date(NA), as.Date(name5, "%Y%m%d"))) %>% # Format the dates
  mutate(t2m = if_else(is.na(name5), as.numeric(NA), as.numeric(value))) %>% # Extract the measured values
  select(lat, lng, date, t2m) %>%
  filter(!is.na(lat) & !is.na(lng) |
           (!is.na(date) & !is.na(t2m)))
## This removes stand alone coordinates (only X or XY) without T2M values
while(length(ind <- which(is.na(weather.JSON$lat))) > 0){
  weather.JSON$lat[ind] <- weather.JSON$lat[ind -1]
}
while(length(ind <- which(is.na(weather.JSON$lng))) > 0){
  weather.JSON$lng[ind] <- weather.JSON$lng[ind -1]
}
## Remove empty rows without T2M values
weather.JSON <- weather.JSON %>%
  filter(!is.na(date) & !is.na(t2m))
## Create an sf object and assign CRS
weather.JSON <- st_as_sf(weather.JSON, coords = c("lng", "lat"), crs = 4326)

In [ ]:
## Subset the data and start preparing for a RasterStack
sep2017Weather <- weather.JSON1 %>%
  filter(date > '2017-09-01' & date < '2017-10-04')
## From long-format table to wide-format table
sep2017Weather <- sep2017Weather %>%
  spread(date, t2m)
## Re-format column labels
colnames(sep2017Weather) <- gsub('-', '_', colnames(sep2017Weather), fixed=TRUE)
## Create RasterStack
st <- raster::stack()
for(i in names(sep2017Weather)[!grepl("['geometry]", names(sep2017Weather))]){
  vals <- pull(sep2017Weather, i)
  ras <- raster::rasterize(sep2017Weather, raster::raster(), vals)
  st <- raster::stack(st,ras)
  remove(vals,ras)
}